<a href="https://colab.research.google.com/github/NotARectangle/Honours2021/blob/dataTest/GPUTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving TNGScriptUncut.json to TNGScriptUncut.json


In [2]:
!pip install transformers==4.2.2

     |████████████████████████████████| 1.8MB 8.2MB/s 
     |████████████████████████████████| 890kB 41.2MB/s 
     |████████████████████████████████| 2.9MB 46.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3bfc30f18abcb579bcf1a9f3d5600bdac2334c4469d8e6bf34ae004e8b14305d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!pip install sklearn

In [4]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 24kB/s 
     |████████████████████████████████| 12.8MB 242kB/s 
     |████████████████████████████████| 7.6MB 4.7MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [5]:
import torch
torch.cuda.is_available()

True

In [17]:
import json
import re
import numpy as np
from sklearn.model_selection import train_test_split

data = json.load(open('TNGScriptUncut.json', 'r'))

script = " "
episodes = data.keys()
#print(data['episode 0'])
for ep in episodes: 
       script += data[ep]

print(len(script))
#print(data.keys())
def build_text_files(data_json, dest_path):
     f = open(dest_path, 'w', encoding='utf-8')
     print("First line of script" + data_json[0])
     data = ''
    # episodes = data_json.keys()
     #for text in episodes:
     for line in data_json:
     #       summary = str(texts)
           # summary = re.sub(r"\s", " ", summary)
     #       data += summary
    # print(str(len(data)))
     f.write(data)

train, test = train_test_split(script,test_size=0.15)




build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


5403403
First line of scriptK
First line of scriptu
Train dataset length: 4592892
Test dataset length: 810511


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

from transformers import TextDataset, DataCollatorForLanguageModeling


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128)

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator


train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

print(len(train_dataset))
print(len(test_dataset))


0
0


/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")
model.to('cuda')

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,
    data_collator=data_collator,            # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

trainer.save_model()

In [ ]:
from transformers import pipeline

picard = pipeline('text-generation',model='./results', tokenizer='microsoft/DialoGPT-small',config={'max_length':800})

#result 

In [ ]:
picard("make it so")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'make it so that the server can run the application. It is not specified, but it is specified with the --test option. This would make it so that the server can get the necessary information so that it can use the service of the database for'}]

In [ ]:
picard("Would you object to your captain")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Would you object to your captain's attitude?\n\nNathan: Yes. I'm not at all ashamed of it. It felt very well-deserved. I never said it was an apology. It was something I intended to do, and"}]

In [ ]:
picard("Doctor crusher")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Doctor crusher of the week!\n\nYou know, I just don't know what's going on, and because I know I did a really good job getting it out, I think there's really been a growing consensus around the state of California"}]